In [1]:
import cv2 #Video and Image processing
import numpy as np #Array manipulation

In [2]:
#pre-trained Haar cascades for face and eye detection
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + './haarcascade_frontalface_default.xml')
eye_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + './haarcascade_eye.xml')

In [3]:
#detect faces and swap eyes in a given frame
def swap_frame(frame):
    #convert the frame to grayscale for haar cascades
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    #detect faces in the frame
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.3, minNeighbors=5)
    
    #process each detected face
    for (x, y, w, h) in faces:
        #Extract the face region from the grayscale and colored frames
        face_roi_gray = gray[y:y + h, x:x + w]
        face_roi_color = frame[y:y + h, x:x + w]
        
        #Detect eyes within the face 
        eyes = eye_cascade.detectMultiScale(face_roi_gray, scaleFactor=1.1, minNeighbors=10, minSize=(30, 30))
        
        #Make sure that 2 eyes were detected
        if len(eyes) == 2:
            #Sort by x-coordinate to distinguish between left & right eye
            eyes = sorted(eyes, key=lambda ex: ex[0])
            
            #Extract the eye region
            eye_1 = eyes[0] #left eye
            eye_2 = eyes[1] #right eye
            
            #Get coordinates and dimensions of each eye
            ex1, ey1, ew1, eh1 = eye_1
            ex2, ey2, ew2, eh2 = eye_2
            
            #crop the eye regions from the face ROI
            left_eye = face_roi_color[ey1:ey1 + eh1, ex1:ex1 + ew1]
            right_eye = face_roi_color[ey2:ey2 + eh2, ex2:ex2 + ew2]
            
            #Resize the eyes to match target ROI
            right_eye_resized = cv2.resize(right_eye, (ew1, eh1))
            left_eye_resized = cv2.resize(left_eye, (ew2, eh2))
            
            #Swap the eyes in the face ROI
            face_roi_color[ey1:ey1+eh1, ex1:ex1+ew1] = right_eye_resized
            face_roi_color[ey2:ey2 + eh2, ex2:ex2 + ew2] = left_eye_resized
            
    return frame
            

In [4]:
#Real-time feed
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        print('Failed to capture frame')
        break
    
    frame = swap_frame(frame)
    cv2.imshow('Eye Swap', frame)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
        
cap.release()
cv2.destroyAllWindows()